In [1]:
import random
import ipyparallel as ipp

##### Declaring the number of nodes

In [2]:
n = 6

##### Generating node UIDs

In [3]:
uids = random.sample(range(1, (n * 10) + 1), n)

print(f"Generated {n} UIDs from the range [1, {n * 10}]")
print("Min UID:", min(uids))
print("Max UID:", max(uids))

Generated 6 UIDs from the range [1, 60]
Min UID: 17
Max UID: 60


##### Connecting to the engines

In [4]:
cluster = ipp.Cluster.from_file()
rc = cluster.connect_client_sync()
rc.wait_for_engines(n)

print(f"Number of available engines: {len(rc.ids)}")

100%|██████████| 6/6 [00:03<00:00,  1.93engine/s]
Number of available engines: 6


#### 'Hello, World!' message from nodes

In [5]:
def hello_world(params):
    uid = params[0]
    n = params[1]
    return f"Hello, World! UID: {uid}, Total Nodes: {n}"

In [6]:
params = zip(uids, [n] * n)

results = rc[:].map_sync(hello_world, params)
print("\n".join(results))

Hello, World! UID: 30, Total Nodes: 6
Hello, World! UID: 60, Total Nodes: 6
Hello, World! UID: 29, Total Nodes: 6
Hello, World! UID: 17, Total Nodes: 6
Hello, World! UID: 50, Total Nodes: 6
Hello, World! UID: 33, Total Nodes: 6


#### LCR algorithm

In [8]:
def setup_communication(id, prev_id, next_id):
    import time
    from ipyparallel import Client

    rc = Client()
    dw = rc[:]

    def send_prev(data):
        dw.push({'incoming_data': data}, block=False, targets=[prev_id])

    def send_next(data):
        # rc[next_id].push({'incoming_data': data}, block=False, targets=[next_id])
        dw.push({'incoming_data': data}, block=False, targets=[next_id])

    def wait_for_data():
        return dw.pull('incoming_data', block=True, targets=[id])

    return send_prev, send_next, wait_for_data

In [9]:
@ipp.require(setup_communication)
def ring_communication(params):
    uid, id, prev_id, next_id = params
    send_prev, send_next, wait_for_data = setup_communication(id, prev_id, next_id)
    send_next(f"Message from f{uid}")
    a = wait_for_data()
    return a

In [ ]:
ids = rc.ids
prev_ids = [(id - 1) % n for id in ids]
next_ids = [(id + 1) % n for id in ids]
params = zip(uids, ids, prev_ids, next_ids)

results = rc[:].map_sync(ring_communication, params)
print("\n".join(results))

In [ ]:
# def lcr(uid):
#     u = uid
#     send = uid
#     status = 'unknown'
#
#     while status == 'unknown':
#         ...

In [ ]:
# results = rc[:].map_sync(lcr, uids)
# print("\n".join(results))

#### HS algorithm

In [ ]:
# TODO